In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import math

import plotly.express as px

pd.options.display.max_rows = 999999
pd.options.display.max_columns = 99999

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

#import sys
#sys.path.insert(0, 'C:/Users/Sean/Documents/python/ufc/scripts/functions/')

from functions import clean_entry_name, melt_crosstab, cleanup_mlb_lineup_data, create_points_own_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# INPUTS

In [10]:
# Setup paths
data_dir = 'C:/Users/ahote/Documents/dk_slate_study_tool/data/'
file_name = '25dollar_single_entry_500entries.csv'

file_path = data_dir + file_name

In [11]:
dk_users = ['Awesemo', 'giantsquid', 'bkreider', 'dacoltz', 'getloose', 'totoroll33', 'BigT44', 'I_Slewfoot_U', 'B_Heals152', 'thepickler']

# READ DATA

In [12]:
# Read in data
df = pd.read_csv(file_path)
print(len(df))

564


In [13]:
df.head()

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,Unnamed: 6,Player,Roster Position,%Drafted,FPTS
0,1,2766590066,bfd77,0,147.10,P Adam Wainwright P Jordan Lyles 1B Vladimir G...,NaN,Lance McCullers Jr.,P,68.26%,13.85
1,2,2766137098,TheHumanCespedes,0,144.90,P Julio Urías P Jordan Lyles 1B Vladimir Guerr...,NaN,Dylan Cease,P,33.51%,9.85
2,3,2766338839,KellyKrobar,0,134.75,P Lance McCullers Jr. P Matt Manning OF Nelson...,NaN,Adam Wainwright,P,30.32%,19.55
3,4,2766619694,ahertz33,0,132.80,P Julio Urías P Kyle Hendricks 1B Vladimir Gue...,NaN,Bryce Harper,OF,28.90%,11.00
4,5,2766466496,status78,0,131.70,P Lance McCullers Jr. P Dylan Cease 1B Vladimi...,NaN,Josh Harrison,3B,28.37%,7.00


# PREP DATA

## We split the master data into 2 different datasets that in tandem will make the final output (have to do this because its basically 2 dataframes in a single csv side-by-side

## Dataset #1 - Player Points Scored and Contest Ownership
## Dataset #2 - Every Contest Entry Name and Lineup 

In [14]:
# Create the 1st dataset
points_own_df = pd.DataFrame()

# Add the main datapoints
points_own_df['player'] = df.Player.dropna()
points_own_df['position'] = df['Roster Position'].dropna() 
# Need to clean this up a bit, the percentages are coming in as strings from the file so we convert to a float
# Strip the percentage sign from the last char and then cast
points_own_df['ownership'] = [float(ownership[:-1]) for ownership in  df['%Drafted'].dropna()]
points_own_df['points'] = df['FPTS'].dropna()

# Now clean the player name a bit - THIS SHOULDN'T BE NECESSARY NOW
#points_own_df['player'] = points_own_df['player'].apply(lambda row: clean_player_name(row)) 

print(len(points_own_df))
points_own_df.head()

154


,player,position,ownership,points
0,Lance McCullers Jr.,P,68.26,13.85
1,Dylan Cease,P,33.51,9.85
2,Adam Wainwright,P,30.32,19.55
3,Bryce Harper,OF,28.90,11.00
4,Josh Harrison,3B,28.37,7.00


In [15]:
# First thing to do is drop the nans from the Lineup field - these are empty lineups that people submitted and should not be included in this analysis
raw_lineup_data = df[['Rank','EntryId','EntryName','Points','Lineup']]
raw_lineup_data = raw_lineup_data.dropna()

In [16]:
raw_lineup_data.head()

,Rank,EntryId,EntryName,Points,Lineup
0,1,2766590066,bfd77,147.10,P Adam Wainwright P Jordan Lyles 1B Vladimir G...
1,2,2766137098,TheHumanCespedes,144.90,P Julio Urías P Jordan Lyles 1B Vladimir Guerr...
2,3,2766338839,KellyKrobar,134.75,P Lance McCullers Jr. P Matt Manning OF Nelson...
3,4,2766619694,ahertz33,132.80,P Julio Urías P Kyle Hendricks 1B Vladimir Gue...
4,5,2766466496,status78,131.70,P Lance McCullers Jr. P Dylan Cease 1B Vladimi...


In [17]:
# Clean up the raw dataframe, parse out the lineups, clean the username, etc..
agg_lineups = cleanup_mlb_lineup_data(raw_lineup_data)

agg_lineups.head()

,Rank,EntryId,EntryName,Points,P1,P2,C,1B,2B,3B,SS,OF1,OF2,OF3
0,1,2766590066,bfd77,147.10,Adam Wainwright,Jordan Lyles,Vladimir Guerrero Jr.,Tim Anderson,Nolan Arenado,Teoscar Hernández,Max Kepler,Leury García,Kiké Hernández,Tres Barrera
1,2,2766137098,TheHumanCespedes,144.90,Julio Urías,Jordan Lyles,Vladimir Guerrero Jr.,Xander Bogaerts,George Springer,J.D. Martinez,Wilmer Flores,Kiké Hernández,Zack Collins,Hunter Renfroe
2,3,2766338839,KellyKrobar,134.75,Lance McCullers Jr.,Matt Manning,Nelson Cruz,Tim Anderson,J.T. Realmuto,Josh Donaldson,Jorge Polanco,Miguel Sanó,Max Kepler,Danny Santana
3,4,2766619694,ahertz33,132.80,Julio Urías,Kyle Hendricks,Vladimir Guerrero Jr.,Bo Bichette,Teoscar Hernández,Robbie Grossman,Kiké Hernández,Akil Baddoo,Danny Jansen,Josh Harrison
4,5,2766466496,status78,131.70,Lance McCullers Jr.,Dylan Cease,Vladimir Guerrero Jr.,Carlos Correa,J.D. Martinez,Wilmer Flores,Kiké Hernández,Curt Casali,Greg Allen,Estevan Florial


# PROCESS DATA

# Here we need to transform the data by melting and creating a crosstab table to show exposures for every user

In [19]:
points_own_df = create_points_own_df(df)

points_own_df.head()

,player,position,ownership,points
0,Lance McCullers Jr.,P,68.26,13.85
1,Dylan Cease,P,33.51,9.85
2,Adam Wainwright,P,30.32,19.55
3,Bryce Harper,OF,28.90,11.00
4,Josh Harrison,3B,28.37,7.00


In [38]:
# Loop through each user and create a dictionary with their data
user_data_dict = {}

for user in dk_users:
    user_data_dict[user] = melt_crosstab(agg_lineups, user)
    #user_data_dict[user]['F'] = user_data_dict[user][['F1','F2','F3','F4','F5','F6']].sum(axis=1)
    user_data_dict[user] = user_data_dict[user][['player','count','exposure']]

KeyError: "['player'] not in index"

In [39]:
# Aggregate the various dataframes into a single one

agg_exposures = pd.DataFrame()

for user in dk_users:
    if user == dk_users[0]:
        agg_exposures = user_data_dict[user][['player','exposure']].round(2)
        agg_exposures.rename(columns={'exposure':user}, inplace=True)
    else:
        agg_exposures = pd.merge(agg_exposures, user_data_dict[user][['player','exposure']].round(2), how='outer', on='player')
        agg_exposures.rename(columns={'exposure':user}, inplace=True)
        
    agg_exposures = agg_exposures.replace(np.nan, 0.0)    
    
agg_exposures.head(25)

KeyError: "['player'] not in index"

In [45]:
# Now merge the 2 datasets that we've created together into 1
master_df = pd.merge(agg_exposures, points_own_df, on='player')
non_user_cols = ['player','position','points', 'ownership']
master_df = master_df[[*non_user_cols, *master_df.columns.difference(non_user_cols)]]

print(len(master_df))

56


# OUTPUT

In [17]:
master_df.sort_values('ownership', ascending=False)

,player,position,points,ownership,MMAdamMartin,Packersfan203,ahotebrown,gehrenberg
11,Leon Edwards,F,90.97,48.22,43.33,29.33,60.00,52.78
17,Deiveson Figueiredo,F,19.10,42.29,34.00,10.00,34.67,27.78
1,Jamahal Hill,F,2.39,39.33,44.00,50.00,45.33,52.78
24,Israel Adesanya,F,84.68,36.53,40.67,0.00,25.33,0.00
3,Brandon Moreno,F,97.27,34.48,32.00,42.00,35.33,47.22
0,Marvin Vettori,F,62.25,33.38,48.00,65.33,30.00,68.06
10,Luigi Vendramini,F,27.47,26.75,20.00,29.33,25.33,58.33
2,Matt Frevola,F,0.00,25.62,1.33,49.33,45.33,31.94
6,Chase Hooper,F,51.05,24.87,20.00,32.00,19.33,0.00
5,Joanne Calderwood,F,55.38,24.40,2.67,35.33,30.00,61.11


In [ ]:
master_df.sort_values('ownership', ascending=False).to_csv('sample_master_df.csv',index=False)